In [1]:
import pandas as pd
import requests
from datetime import datetime
import json
import os
from dateutil import parser
import collections

The following code helps to determine whether new data is inserted in api database or not, if so then we have to pull new movies data and put that in database and update tracking parameters. 

In [2]:
last_page =  requests.get('https://yts.torrentbay.to/api/v2/list_movies.json?page=821&limit=50').json()
first_page = requests.get('https://yts.torrentbay.to/api/v2/list_movies.json?page=1&limit=50').json()

In [3]:
movie_a = last_page['data']['movies'][1]

In [7]:
movie_a

{'id': 1336,
 'url': 'https://yts.torrentbay.to/movies/gone-with-the-wind-1939',
 'imdb_code': 'tt0031381',
 'title': 'Gone with the Wind',
 'title_english': 'Gone with the Wind',
 'title_long': 'Gone with the Wind (1939)',
 'slug': 'gone-with-the-wind-1939',
 'year': 1939,
 'rating': 8.2,
 'runtime': 238,
 'genres': ['Action', 'Drama', 'History', 'Romance', 'War'],
 'summary': 'This tale of the Old South from the start of the Civil War through to the period of reconstruction focuses on the beautiful Scarlett O&#39;Hara. Before the start of the war life at the O&#39;Hara plantation, Tara, could only be described as genteel. As for the young Scarlett, she is without doubt the most beautiful girl in the area and is always the belle of the ball. She is very much looking forward to a barbecue at the nearby Wilkes plantation as she will get to see the man she loves, Ashley Wilkes. She is more than a little dismayed when she hears that he is to marry his cousin Melanie Hamilton and in a fit 

In [ ]:
movie_a['genres']

In [ ]:
ids = [41763,41713]
ids

In [ ]:
def check_new_movies(ids:list):

    page_number = 1
    movies_list = []
    while True:
        init_page = requests.get(f'https://yts.torrentbay.to/api/v2/list_movies.json?page={page_number}&limit=50').json()

        current_ids = [init_page['data']['movies'][0]['id'],init_page['data']['movies'][-1]['id']]

        if current_ids > ids:
            for movie in init_page['data']['movies']:
                movies_list.append(movie['title'])
        
            page_number = page_number + 1

        else:
            break

    return movies_list

In [ ]:
mov_list = check_new_movies(ids)

In [ ]:
for movie in first_page['data']['movies']:
    print(movie)

Below code is for extracting movie cast based on "IMDB code"

In [10]:
import requests
from bs4 import BeautifulSoup
import re
import time

In [21]:
url = f'https://www.imdb.com/title/tt2065946/fullcredits'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')

In [22]:
table = soup.find('table', attrs={'class' : 'cast_list'})

In [23]:
cast = table.find_all('a')

In [24]:
cast[0].get_text().strip('\n').strip()

''

In [63]:
cast

[<a href="/name/nm1881049/"><img alt="Miho Fujima" class="" height="44" src="https://m.media-amazon.com/images/S/sash/N1QWYSqAfSJV62Y.png" title="Miho Fujima" width="32"/></a>,
 <a href="/name/nm1881049/"> Miho Fujima
 </a>,
 <a href="/name/nm0620267/"><img alt="Yûko Nakamura" class="" height="44" src="https://m.media-amazon.com/images/S/sash/N1QWYSqAfSJV62Y.png" title="Yûko Nakamura" width="32"/></a>,
 <a href="/name/nm0620267/"> Yûko Nakamura
 </a>,
 <a href="/name/nm0913831/"><img alt="Makiko Watanabe" class="loadlate hidden" height="44" loadlate="https://m.media-amazon.com/images/M/MV5BMTY4MDAwNzYxMF5BMl5BanBnXkFtZTcwNjQxNDc3MQ@@._V1_UX32_CR0,0,32,44_AL_.jpg" src="https://m.media-amazon.com/images/S/sash/N1QWYSqAfSJV62Y.png" title="Makiko Watanabe" width="32"/></a>,
 <a href="/name/nm0913831/"> Makiko Watanabe
 </a>]

In [64]:
re.findall(r'title="(.*?)"', str(cast))

['Miho Fujima', 'Yûko Nakamura', 'Makiko Watanabe']

In [62]:
cast_list = []
for actor in cast:
    print(re.findall(r'title="(.*?)"', str(actor)))

['Miho Fujima']
[]
['Yûko Nakamura']
[]
['Makiko Watanabe']
[]


In [ ]:
tuple(cast_list[::3])

## method to get cast list of given imdb movie id

In [65]:
def get_cast(imdb_id : str):
    
    url = f'https://www.imdb.com/title/{imdb_id}/fullcredits'
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    table = soup.find('table', attrs={'class' : 'cast_list'})
    cast = table.find_all('a')   
    
    return re.findall(r'title="(.*?)"', str(cast))

In [69]:
%%time
ac = get_cast('tt2065942')

Wall time: 15.2 s


In [70]:
ac

['Atomic Kitten',
 'Irene Blanco',
 'Sandra Blanco',
 'Miguel Ángel Cabrera',
 'María del Mar',
 'Natalia Estrada',
 'Rosario Flores',
 'Pao González',
 'Sandra Gálvez',
 'Sonia Gálvez',
 'Dioni Martín',
 'Ángeles Muñoz',
 'María Rodríguez',
 'Pedro Ruiz',
 'Gala Évora']

{'status': 'ok',
 'status_message': 'Query was successful',
 'data': {'movie_count': 42330,
  'limit': 50,
  'page_number': 1,
  'movies': [{'id': 43021,
    'url': 'https://yts.torrentbay.to/movies/men-2022',
    'imdb_code': 'tt13841850',
    'title': 'Men',
    'title_english': 'Men',
    'title_long': 'Men (2022)',
    'slug': 'men-2022',
    'year': 2022,
    'rating': 6.3,
    'runtime': 100,
    'genres': ['Drama', 'Horror', 'Sci-Fi'],
    'summary': 'A young woman goes on a solo vacation to the English countryside following the death of her ex-husband.',
    'description_full': 'A young woman goes on a solo vacation to the English countryside following the death of her ex-husband.',
    'synopsis': 'A young woman goes on a solo vacation to the English countryside following the death of her ex-husband.',
    'yt_trailer_code': 'VrYNHYX1qc0',
    'language': 'en',
    'mpa_rating': 'R',
    'background_image': 'https://yts.torrentbay.to/assets/images/movies/men_2022/background.jp

###  The following data needed to be pulled from api response and put in warehouse
#### : imdb_code -> unique movie code given by IMDB
#### : title_long -> full movie title with year
#### : year -> released year
#### : rating -> movie rating (1-10)
#### : runtime -> in minutes
#### : genres -> can be one or more
#### : summary -> summary of movie
#### : cast -> need to scrape from IMDB
#### : mpa_rating -> movie sensorship rating
#### : language -> can be one or more

In [4]:
movie_a['imdb_code'], movie_a['title_long'], movie_a['year'], movie_a['rating'], movie_a['runtime'], (movie_a['genres']), movie_a['mpa_rating'], movie_a['language']

('tt0031381',
 'Gone with the Wind (1939)',
 1939,
 8.2,
 238,
 ['Action', 'Drama', 'History', 'Romance', 'War'],
 '',
 'en')

Here is the main problem, as movie has multiple genres, we need to figure out how to insert genres without repetation.
First we need to insert "imdb_code" and "movie_title" in movies table. Then we have to check that genre exists in genre table, if so then we can skip, else insert it.

In [5]:
movie_a['genres'][0]

'Action'